In [9]:
# Please load this first

library(tidyverse)
library(repr)

In [12]:
# Run this to load the data sets and display respective summary statistics

players <- read_csv("data/players.csv")
sessions <- read_csv("data/sessions.csv")

summarized_players <- summarize(players, mean_age = mean(Age, na.rm = TRUE), mean_played_hours = mean(played_hours))

summarized_players
# summary_sessions



Rows: 196 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, Age
lgl (1): subscribe

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


mean_age,mean_played_hours
<dbl>,<dbl>
21.13918,5.845918


### Data Description

##### Data file 1 - players.csv
- 196 rows (observations) and 7 columns (variables)
- Each row corresponds to a unique player
    - experience (character)
        - This variable indicates the experience level of each MineCraft player. There are five unique experience levels: "Beginner", "Amateur", "Regular", "Pro" and "Veteran". These variables could be changed to be of the 'factor' data type.
    - subscribe (factor)
      - This variable contains TRUE or FALSE logical values, indicating whether a player is subscribed to a gaming newsletter.
    - hashedEmail (character)
      - This variable contains the player's email addresses, hashed for privacy.
    - played_hours (double)
      - This variable indicates the total number of hours played on the MineCraft server by each player.
      - The mean value of played_hours is 5.85.
    - name (character)
      - This variable reports the first name of each player.
    - gender (character)
      - This variable reports the gender identity of each player. This variable's data type could be changed from 'character' to 'factor.
    - Age (double)
      - This variable contains the age of each player. This variable's data type could be changed from 'double' to 'integer'.
      - The mean value of Age is 21.14.

##### Data file 2 - sessions.csv
- 1535 rows (observations) and 5 columns (variables)
- Each row corresponds to a unique play session.
    - hashedEmail (character)
      - This variable contains the player's email addresses, hashed for privacy.
    - start_time (character)
      - Describes the start date and time of a player's play session.
    - end_time (character)
      - Describes the end date and time of a player's play session.
    - original_start_time (double)
      - Contains the start time of a player's play session in UNIX timestamp form.
    - original_end_time (double)
      - Contains the end time of a player's play session in UNIX timestamp form.
     
For most play sessions, the original_start_time and original_end_time values appear the same as the UNIX timestamp values were not recorded to include enough decimal points, so the difference of the two values often won't communicate the total play time of a session.

##### Question:
For this project, I will be looking into the following question of interest: "What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?" Using this topic, I plan to determine if 'Age' and 'played_hours' can accurately predict 'subscribe' in 'players.csv'. 